## Template matching using FLANN and SIFT

In [4]:
import cv2
import numpy as np

In [5]:
def extract_features(image, bbox, feature_extractor):
    """ Extract features from the specified bounding box in the image. """
    x, y, w, h = bbox
    roi = image[y:y+h, x:x+w]
    keypoints, descriptors = feature_extractor.detectAndCompute(roi, None)
    return keypoints, descriptors, roi

In [6]:
def match_features(descriptors1, descriptors2):
    """ Match features using FLANN based matcher. """
    FLANN_INDEX_KDTREE = 0
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=50)

    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(descriptors1, descriptors2, k=2)
    # Filter matches using the Lowe's ratio test
    good_matches = []
    for m, n in matches:
        if m.distance < 0.7*n.distance:
            good_matches.append(m)
    return good_matches

In [ ]:
def find_bbox_shift(good_matches, keypoints1, keypoints2):
    """ Calculate the shift in bounding box based on matched features. """
    if not good_matches:
        return 0, 0  # No shift if no good matches found

    dxs = []
    dys = []
    for match in good_matches:
        pt1 = keypoints1[match.queryIdx].pt
        pt2 = keypoints2[match.trainIdx].pt
        dxs.append(pt2[0] - pt1[0])
        dys.append(pt2[1] - pt1[1])
    shift_x = np.mean(dxs)
    shift_y = np.mean(dys)
    return int(shift_x), int(shift_y)


In [ ]:
# Load images
image1 = cv2.imread('frame1.jpg')
image2 = cv2.imread('frame2.jpg')

# Define bounding boxes
bbox1 = (100, 100, 50, 50)  # x, y, width, height
bbox2 = (95, 95, 60, 60)  # Slightly larger and extended bounding box

# Create SIFT feature extractor
sift = cv2.SIFT_create()

# Extract features from both images
keypoints1, descriptors1, _ = extract_features(image1, bbox1, sift)
keypoints2, descriptors2, _ = extract_features(image2, bbox2, sift)

# Match features
good_matches = match_features(descriptors1, descriptors2)

# Find bounding box shift
shift_x, shift_y = find_bbox_shift(good_matches, keypoints1, keypoints2)
print(f"Suggested bounding box shift: {shift_x} px in x direction, {shift_y} px in y direction")
